### Fakeddit

### Setup Environment:
* [Paper](https://arxiv.org/pdf/1911.03854.pdf)

In [1]:
%cd ..

/home/datascience/Data Fusion


In [2]:
import os
import pandas as pd

from src.classifiers import preprocess_data, process_labels,split_data

from src.classifiers import VQADataset
from torch.utils.data import DataLoader

from src.classifiers_cpu_metrics import calculate_memory

In [3]:
PATH = 'Embeddings_vlm/fakeddit/'
FILE = 'embeddings_clip.csv'

FILE_PATH = os.path.join(PATH, FILE)

## Get data

In [4]:
df = pd.read_csv(FILE_PATH)
df.drop(columns=['hasImage', 'image_url'], inplace=True)
df.head()

,author,clean_title,created_utc,domain,id,linked_submission_id,num_comments,score,subreddit,title,...,text_embedding_502,text_embedding_503,text_embedding_504,text_embedding_505,text_embedding_506,text_embedding_507,text_embedding_508,text_embedding_509,text_embedding_510,text_embedding_511
0,michaelconfoy,loose talk got there first keep it under your ...,1.429942e+09,i.imgur.com,datasets/fakeddit/images/33snxf.jpg,NaN,6.0,70,propagandaposters,"""Loose talk got there first! Keep it under you...",...,0.017499,0.040466,0.121275,-0.046296,0.051412,0.016091,0.010176,0.065790,-0.091243,-0.035023
1,SaltMineForeman,this tree at my campsite naturally grew a hear...,1.554421e+09,i.imgur.com,datasets/fakeddit/images/b9k2ta.jpg,NaN,2.0,30,mildlyinteresting,This tree at my campsite naturally grew a hear...,...,-0.035342,0.014651,0.022642,-0.005436,-0.041879,0.019345,-0.020924,0.012200,-0.047352,0.045813
2,Shawn_666,fdr addressing the nation,1.522982e+09,i.redd.it,datasets/fakeddit/images/8a5wy9.jpg,NaN,1.0,12,fakehistoryporn,FDR addressing the nation (1932),...,-0.027288,0.020853,-0.003959,0.000404,0.034326,-0.029220,-0.002929,-0.065850,0.016998,0.019142
3,YOLO2THEMAX,face morph,1.448948e+09,NaN,datasets/fakeddit/images/cxiu2qb.jpg,3urf4a,NaN,2,psbattle_artwork,Face morph,...,-0.011685,-0.003866,0.037784,0.010292,-0.004787,0.003773,-0.012201,-0.028986,-0.010641,-0.032186
4,FarMojo,a japanese kamikaze pilot of the sixth air arm...,1.559169e+09,i.redd.it,datasets/fakeddit/images/bul96p.jpg,NaN,0.0,8,fakehistoryporn,A Japanese kamikaze pilot of the sixth air arm...,...,0.021707,0.011216,0.022868,-0.023618,-0.039166,-0.000059,0.033039,-0.088854,0.024785,-0.023957


## Data Perparation

In [5]:
# Split the data
train_df, test_df = split_data(df)

# Select features and labels vectors
text_columns = [column for column in df.columns if 'text' in column] #[f'text_{i}']
image_columns = [column for column in df.columns if 'image' in column] #[f'image_{i}']
label_columns = '2_way_label'


# Process and one-hot encode labels for training set
train_labels, mlb, train_columns = process_labels(train_df, col=label_columns)
test_labels = process_labels(test_df, col=label_columns, train_columns=train_columns)


train_dataset = VQADataset(train_df, text_columns, image_columns, label_columns, mlb, train_columns)
test_dataset = VQADataset(test_df, text_columns, image_columns, label_columns, mlb, train_columns)


train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

Train Shape: (56214, 1039)
Test Shape: (5911, 1039)


### Models

In [6]:
text_input_size = len(text_columns)
image_input_size = len(image_columns)
if label_columns == '2_way_label':
    output_size = 1
else:
    output_size = len(pd.unique(train_df[label_columns]))
multilabel = False

In [7]:
calculate_memory(train_loader, test_loader, text_input_size, image_input_size, output_size)

Early fusion:
Average Memory per Batch in Train: 0.25 MB
Total Memory Usage per Epoch Train: 219.80 MB (excluding model parameters)
Test:
Average Memory per Batch in Test: 0.03 MB
Total Memory Usage per Epoch Test: 2.45 MB (excluding model parameters)
Model: 
Model Memory Usage: 0.50 MB

Late fusion:
Average Memory per Batch in Train: 0.25 MB
Total Memory Usage per Epoch Train: 219.80 MB (excluding model parameters)
Test:
Average Memory per Batch in Test: 0.03 MB
Total Memory Usage per Epoch Test: 2.45 MB (excluding model parameters)
Model: 
Model Memory Usage: 0.25 MB
